## Let's discuss the basics first

### Tensors & Gradients

In [1]:
import torch
import numpy as np

In [2]:
print(torch.__version__)
print(np.__version__)

1.1.0
1.16.2


### A tensor is an n-dimensional array.

In [3]:
# Create tensors.
X = torch.tensor(2.)
W = torch.tensor(3., requires_grad=True)
b = torch.tensor(4., requires_grad=True)
X, W, b

(tensor(2.), tensor(3., requires_grad=True), tensor(4., requires_grad=True))

In [4]:
# Arithmetic operations
y = W * X + b
print(y)

tensor(10., grad_fn=<AddBackward0>)


In PyTorch, we can automatically compute the derivative of y w.r.t. a tensor that has "requires_grad set to True" e.g. w and b here.

In [5]:
# Compute gradients
y.backward()

In [6]:
# Display gradients
print('dy/dw:', W.grad)
print('dy/db:', b.grad)

dy/dw: tensor(2.)
dy/db: tensor(1.)


# Let's solve a toy problem using toy data

## Problem Formulation
We'll create a model that predicts product sells for iphone (target variables) by looking at the average imcome, price and tax (input variables or features) in a country. 

## Model architecture:
Our model architecture (or equation) is y = WX + b. 
This architecture will create a linear regression model, i.e. each target variable is estimated to be a weighted sum of the input variables (w*x), offset by some constant, known as a bias (b).

## Objective: 
We want to find a suitable set of weights (W) and a bias (b) using the training data, to make accurate predictions (i.e. reduce loss in predictions).

In [12]:
# Input (salary in K, price in K, tax in K)
inputs = np.array([[70., 0.1, 0.043], 
                   [90., 0.8, 0.064], 
                   [80., 0.13, 0.058], 
                   [100., 0.4, 0.037], 
                   [60., 0.9, 0.7]], dtype='float32')

In [13]:
# Targets iphone sell number in K
targets = np.array([5, 8, 12, 2, 1], dtype='float32')

PyTorch can work with only PyTorch tensors. Therefore, we need to convert inputs and targets to PyTorch tensors.

In [14]:
# Convert inputs and targets to tensors
X = torch.from_numpy(inputs)
y = torch.from_numpy(targets)
print(inputs)
print(targets)

[[7.0e+01 1.0e-01 4.3e-02]
 [9.0e+01 8.0e-01 6.4e-02]
 [8.0e+01 1.3e-01 5.8e-02]
 [1.0e+02 4.0e-01 3.7e-02]
 [6.0e+01 9.0e-01 7.0e-01]]
[ 5.  8. 12.  2.  1.]


### Initialise weights and biases
The weights and biases can be represented as matrices and initialized with random values. The first row of W and the first element of b are use to predict the first target variable i.e. iphone sell, and similarly the second row of W the second element of be are used for second target variable.

In [22]:
# Weights and biases
W = torch.randn(1, 3, requires_grad=True)
b = torch.randn(1, requires_grad=True)
print(W)
print(b)

tensor([[ 0.7765, -0.1135,  0.7029]], requires_grad=True)
tensor([0.4655], requires_grad=True)


The model is simply a function that performs a matrix multiplication of the input X and the weights W (transposed) and adds the bias b (replicated for each observation).

In [23]:
# Define the architecture
def model(X):
    return X @ W.t() + b

In [24]:
# Generate predictions
preds = model(X)
print(preds)

tensor([[54.8403],
        [70.3058],
        [62.6125],
        [78.0974],
        [47.4461]], grad_fn=<AddBackward0>)


In [25]:
# Compare with targets
print(y)

tensor([ 5.,  8., 12.,  2.,  1.])


## Loss Function
We can compare the predictions with the actual targets, using the following method:
1. Calculate the difference between the two matrices (preds and targets).
2. Square all elements of the difference matrix to remove negative values.
3. Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the mean squared error (MSE).

In [26]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [27]:
preds

tensor([[54.8403],
        [70.3058],
        [62.6125],
        [78.0974],
        [47.4461]], grad_fn=<AddBackward0>)

The resulting number is called the loss, because it indicates how bad the model is at predicting the target variables. Lower the loss, better the model.

In [28]:
# Compute loss
loss = mse(preds, y)
print(loss)

tensor(3390.0071, grad_fn=<DivBackward0>)


### Compute Gradients
In PyTorch, we can automatically compute the gradients, i.e. derivative of the loss w.r.t. to the weights and biases (because weights and biases have requires_grad set to True).

In [29]:
# Compute gradients
loss.backward()

The gradients are stored in the .grad attribute of the tensors W and b.

In [30]:
# Gradients for weights
print(W)
print(W.grad)

tensor([[ 0.7765, -0.1135,  0.7029]], requires_grad=True)
tensor([[9436.7383,   52.3043,   16.6160]])


In [31]:
# Gradients for bias
print(b)
print(b.grad)

tensor([0.4655], requires_grad=True)
tensor([114.1208])


From calculus we know that the gradient indicates the rate of change of the loss, or the slope of the loss function w.r.t. the weights and biases. 

If a gradient element is postive,
1. increasing the element's value slightly will increase the loss.
2. decreasing the element's value slightly will decrease the loss.

If a gradient element is negative,
1. increasing the element's value slightly will decrease the loss.
2. decreasing the element's value slightly will increase the loss.

Also, we know that the increase or decrease is proportional to the value of the gradient.

In [32]:
# Adjust weights & reset gradients
with torch.no_grad():
    W -= W.grad * 1e-5
    b -= b.grad * 1e-5
    W.grad.zero_()
    b.grad.zero_()

##### Note, we have to reset the gradients to zero before moving forward, because PyTorch accumulates gradients.

In [33]:
W.grad.zero_()
b.grad.zero_()
print(W.grad)
print(b.grad)

tensor([[0., 0., 0.]])
tensor([0.])


## Gradient descent algorithm
Now we will adjust weights and biases using gradient descent. This requires reducing the loss using the gradient descent algorithm, which has the following steps:
1. Generate predictions
2. Calculate the loss
3. Compute gradients w.r.t the weights and biases
4. Adjust the weights and biases by subtracting a small quantity proportional to the gradient
5. Reset the gradients to zero

In [34]:
# Generate predictions
preds = model(X)
print(preds)

tensor([[48.2333],
        [61.8112],
        [55.0619],
        [68.6593],
        [41.7824]], grad_fn=<AddBackward0>)


In [35]:
# Calculate the loss
loss = mse(preds, y)
print(loss)

tensor(2558.1218, grad_fn=<DivBackward0>)


In [36]:
# Compute gradients
loss.backward()

In [37]:
# Adjust weights & reset gradients
with torch.no_grad():
    W -= W.grad * 1e-5
    b -= b.grad * 1e-5
    W.grad.zero_()
    b.grad.zero_()

In [38]:
print(W)
print(b)

tensor([[ 0.6002, -0.1145,  0.7026]], requires_grad=True)
tensor([0.4634], requires_grad=True)


With the new weights and biases, the model should have a lower loss.

In [39]:
# Calculate loss
preds = model(X)
loss = mse(preds, y)
print(loss)

tensor(1931.3949, grad_fn=<DivBackward0>)


## Train for multiple epochs
To reduce the loss further, we repeat the process of adjusting the weights and biases using the gradients multiple times. Each repetation with all training instances is called an epoch.

In [46]:
# Train for 2000 epochs
for i in range(2000):
    preds = model(X)
    loss = mse(preds, y)
    #print(loss)
    loss.backward()
    with torch.no_grad():
        W -= W.grad * 1e-7
        b -= b.grad * 1e-7
        W.grad.zero_()
        b.grad.zero_()

In [47]:
# Calculate loss
preds = model(X)
loss = mse(preds, y)
print(loss)

tensor(16.8275, grad_fn=<DivBackward0>)


In [48]:
# Print predictions
preds

tensor([[4.7970],
        [5.9643],
        [5.4213],
        [6.6095],
        [4.5470]], grad_fn=<AddBackward0>)

In [50]:
# Print targets
y

tensor([ 5.,  8., 12.,  2.,  1.])